In [1]:
import torch
import random
import torchvision
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import transforms
from torch.optim.lr_scheduler import MultiStepLR
from sklearn.model_selection import train_test_split

from utils.util import icarl_cifar100_augment_data, get_dataset_per_pixel_mean, get_image_list, dataset_list, PathDataset

from avalanche.models import IcarlNet, make_icarl_net, initialize_icarl_net
from avalanche.training.supervised import ICaRL
from avalanche.logging import InteractiveLogger, WandBLogger
from avalanche.benchmarks.datasets import CIFAR10
from avalanche.benchmarks.generators import nc_benchmark
from avalanche.benchmarks.utils import AvalancheDataset
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.plugins.lr_scheduling import LRSchedulerPlugin
from avalanche.evaluation.metrics import ExperienceAccuracy, StreamAccuracy, ExperienceForgetting, ExperienceTime, EpochAccuracy

In [2]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed) # if use multi-GPU
cudnn.deterministic = True  # 연산 처리 속도 감소 -> 모델과 코드를 배포해야 하는 연구 후반 단계에 사용
cudnn.benchmark = False

In [3]:
data_path = "../Data/CASIA-WebFace/" 

dataset = get_image_list(data_path)
x_train, x_test, y_train, y_test = train_test_split(dataset[0], dataset[1], test_size=0.2, shuffle=True, random_state=seed, stratify=dataset[1])

train_set = dataset_list(x_train, y_train)
test_set = dataset_list(x_test, y_test)

print("train : ", len(y_train))
print("test : ", len(y_test))

train :  395531
test :  98883


In [4]:
pixel_transforms = transforms.Compose([transforms.ToTensor()])
train_per_pixel_mean = get_dataset_per_pixel_mean(train_set, pixel_transforms)
eval_per_pixel_mean = get_dataset_per_pixel_mean(test_set, pixel_transforms)

transforms_group = dict(
       train=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              lambda img_pattern: img_pattern - train_per_pixel_mean,
              icarl_cifar100_augment_data,
              ]
       ),
       None,
       ),
       eval=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              lambda img_pattern: img_pattern - eval_per_pixel_mean,
              ]
       ),
       None,
       )
)

In [5]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

num_class = 10575
incremental = 5
lr_milestones = [49, 63]
lr_factor = 5.0
fixed_class_order = class_list = list(range(num_class))
random.Random(seed).shuffle(fixed_class_order)

train_set = PathDataset(train_set, transforms_group, "train")
test_set = PathDataset(test_set, transforms_group, "eval")

scenario = nc_benchmark(train_dataset=train_set,
                        test_dataset=test_set,
                        n_experiences=incremental,
                        task_labels=False,
                        seed=seed,
                        shuffle=False,
                        fixed_class_order = fixed_class_order
                        )

model: IcarlNet = make_icarl_net(num_classes=num_class)
model.apply(initialize_icarl_net)
# model = torchvision.models.resnet18(pretrained=False, num_classes=num_class)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=2.0, momentum=0.9, weight_decay=1e-5)
sched = LRSchedulerPlugin(MultiStepLR(optimizer, lr_milestones, gamma=1.0 / lr_factor))

In [6]:
interactive_logger = InteractiveLogger()
wandb_logger = WandBLogger(project_name="Face Recognition", run_name="iCaRL-CASIA-M2000")
eval_plugin = EvaluationPlugin(
    EpochAccuracy(),
    ExperienceAccuracy(),
    ExperienceForgetting(),
    ExperienceTime(),
    StreamAccuracy(),
    loggers=[interactive_logger, wandb_logger])

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hopo55. Use `wandb login --relogin` to force relogin


/home/cal-06/anaconda3/envs/hspark/lib/python3.8/site-packages/avalanche/training/plugins/evaluation.py:81: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  warnings.warn(


In [7]:
memory_size = 2000
train_batch = 10
eval_batch = 1
epoch = 10

buffer_transform = transforms.Compose([icarl_cifar100_augment_data])

strategies = ICaRL(model.feature_extractor, model.classifier, optimizer, memory_size, buffer_transform=buffer_transform, fixed_memory=True, train_mb_size=train_batch, train_epochs=epoch, eval_mb_size=eval_batch, device=device, plugins=[sched], evaluator=eval_plugin)  # criterion = ICaRLLossPlugin()

ModuleAttributeError: 'ResNet' object has no attribute 'feature_extractor'

In [ ]:

for i, exp in enumerate(scenario.train_stream):
    eval_exps = [e for e in scenario.test_stream][: i + 1]
    strategies.train(exp)
    strategies.eval(eval_exps)

-- >> Start of training phase << --
100%|██████████| 8220/8220 [14:23<00:00,  9.52it/s]
Epoch 0 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0057
100%|██████████| 8220/8220 [13:49<00:00,  9.91it/s]
Epoch 1 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0047
100%|██████████| 8220/8220 [14:01<00:00,  9.77it/s]
Epoch 2 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0050
100%|██████████| 8220/8220 [14:23<00:00,  9.52it/s]
Epoch 3 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0055
100%|██████████| 8220/8220 [14:22<00:00,  9.53it/s]
Epoch 4 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0050
100%|██████████| 8220/8220 [14:23<00:00,  9.52it/s]
Epoch 5 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0047
100%|██████████| 8220/8220 [14:25<00:00,  9.50it/s]
Epoch 6 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0049
 30%|███       | 2476/8220 [04:19<10:44,  8.91it/s]

KeyboardInterrupt: 